# Superhero Database Assignment (SQLite & Google Colab)

## Objective
In this assignment, you will design and implement a **Superhero Database** using SQLite in Google Colab. This database will store superheroes, villains, and their battles. You will write SQL queries to analyze battle outcomes and relationships between heroes and villains.

### Group Size:
- 2 to 3 students per group.

### Tools Required:
- **Google Colab**
- **SQLite (built into Python)**
- **CSV files (provided for import)**

## Step 1: Setting Up Your SQLite Database

### Tasks:
1. Initialize SQLite in Google Colab.
2. Create a database named `superhero_db.sqlite`.
3. Define the database schema, ensuring it includes the following tables:
   - `heroes`: Stores superhero names, powers, and teams.
   - `villains`: Stores villains and their evil plans.
   - `battles`: Tracks battles between heroes and villains, including the outcome.
4. Implement primary keys and foreign keys where appropriate.

✅ **Checkpoint:** Ensure that the database and tables have been successfully created.

In [ ]:
import sqlite3

# Connect to a database file instead of in-memory
conn = sqlite3.connect('superhero_db.sqlite')
cursor = conn.cursor()

# Drop existing tables
ddl_statements = '''
DROP TABLE IF EXISTS heroes;
DROP TABLE IF EXISTS villains;
DROP TABLE IF EXISTS battles;
'''
cursor.executescript(ddl_statements)

# Create new tables
ddl_statements = '''
CREATE TABLE heroes (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT NOT NULL,
    power TEXT,
    team TEXT
);

CREATE TABLE villains (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT NOT NULL,
    evil_plan TEXT
);

CREATE TABLE battles (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    hero_id INTEGER,
    villain_id INTEGER,
    outcome TEXT,
    FOREIGN KEY (hero_id) REFERENCES heroes(id),
    FOREIGN KEY (villain_id) REFERENCES villains(id)
);
'''
cursor.executescript(ddl_statements)
conn.commit()

print("Database and tables successfully created.")


Database and tables successfully created.


## Step 2: Import Data from CSV Files

### Tasks:
1. Download the provided CSV files.
2. Upload them to Google Colab.
3. Write a script to import data from these files into the appropriate tables.
4. Verify that the data has been successfully inserted.

✅ **Checkpoint:** Ensure that each table contains the expected data from the CSV files.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd

# Load CSV files into Pandas DataFrames
heroes_df = pd.read_csv("/content/sample_data/heroes.csv")
villains_df = pd.read_csv("/content/sample_data/villains.csv")
battles_df = pd.read_csv("/content/sample_data/battles.csv")

# Insert data into SQLite tables
heroes_df.to_sql("heroes", conn, if_exists="append", index=False)
villains_df.to_sql("villains", conn, if_exists="append", index=False)
battles_df.to_sql("battles", conn, if_exists="append", index=False)

# Commit changes
conn.commit()

print("Data successfully imported into SQLite database.")

Data successfully imported into SQLite database.


## Step 3: Querying the Database

### Tasks:
Write SQL queries to answer the following:
1. Retrieve all superheroes and their powers.
2. Find all battles where the hero won.
3. List all villains and their evil plans.
4. Determine which superhero has fought the most battles.

✅ **Checkpoint:** Ensure that all queries return meaningful results from the database.

In [ ]:
# 1. Retrieve all superheroes and their powers
query1 = "SELECT name, power FROM heroes;"
heroes_df = pd.read_sql(query1, conn)
print("\nAll Superheroes and Their Powers:")
print(heroes_df)

# 2. Find all battles where the hero won
query2 = "SELECT * FROM battles WHERE outcome = 'Hero Wins';"
battles_won_df = pd.read_sql(query2, conn)
print("\nBattles Where the Hero Won:")
print(battles_won_df)

# 3. List all villains and their evil plans
query3 = "SELECT name, evil_plan FROM villains;"
villains_df = pd.read_sql(query3, conn)
print("\nAll Villains and Their Evil Plans:")
print(villains_df)

# 4. Determine which superhero has fought the most battles
query4 = """
SELECT heroes.name, COUNT(battles.hero_id) AS battle_count
FROM battles
JOIN heroes ON battles.hero_id = heroes.id
GROUP BY battles.hero_id
ORDER BY battle_count DESC
LIMIT 1;
"""
top_hero_df = pd.read_sql(query4, conn)
print("\nSuperhero Who Fought the Most Battles:")
print(top_hero_df)


All Superheroes and Their Powers:
             name               power
0      Spider-Man        Web-slinging
1        Iron Man       Powered Armor
2          Batman    Genius Detective
3        Superman      Super Strength
4            Thor      God of Thunder
5    Wonder Woman  Amazonian Strength
6   Black Panther    Enhanced Agility
7       The Flash         Super Speed
8            Hulk      Super Strength
9  Doctor Strange         Mystic Arts

Battles Where the Hero Won:
   id  hero_id  villain_id    outcome
0   2        9           3  Hero Wins
1   5        1           7  Hero Wins
2   7        9           8  Hero Wins
3   8        3           2  Hero Wins
4   9        7           5  Hero Wins
5  13        8           9  Hero Wins
6  20        3           4  Hero Wins
7  29        3           5  Hero Wins
8  30        5           5  Hero Wins

All Villains and Their Evil Plans:
           name                           evil_plan
0  Green Goblin                  Terrorize New Yor

## Step 4: Add a Custom Feature

### Tasks:
1. Design and implement an additional table related to superheroes or villains.
2. Populate it with data.
3. Write at least one query to retrieve meaningful insights from the new table.

**Examples of Custom Features:**
- Sidekicks for superheroes
- Villain hideouts
- Battle locations
- Superhero equipment

✅ **Checkpoint:** Ensure that the new table and queries function correctly.

In [ ]:
# drop sidekicks table if exists
cursor.execute("DROP TABLE IF EXISTS sidekicks")

# Create new table for sidekicks
ddl_statement = '''
CREATE TABLE IF NOT EXISTS sidekicks (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    hero_id INTEGER,
    hero_name TEXT NOT NULL,
    sidekick_name TEXT NOT NULL,
    sidekick_power TEXT,
    FOREIGN KEY (hero_id) REFERENCES heroes(id)
);
'''
cursor.execute(ddl_statement)

# Sidekick data with hero names
sidekick_data = [
    (1, "Spider-Man", "Miles Morales", "Web-slinging"),
    (2, "Iron Man", "War Machine", "Powered Armor"),
    (3, "Batman", "Robin", "Acrobatics"),
    (4, "Superman", "Supergirl", "Super Strength"),
    (5, "Thor", "Beta Ray Bill", "God of Thunder"),
    (6, "Wonder Woman", "Donna Troy", "Amazonian Strength"),
    (7, "Black Panther", "Shuri", "Enhanced Intelligence"),
    (8, "The Flash", "Kid Flash", "Super Speed"),
    (9, "Hulk", "She-Hulk", "Super Strength"),
    (10, "Doctor Strange", "Wong", "Mystic Arts"),
]

# Insert data into the sidekicks table
cursor.executemany('''
    INSERT INTO sidekicks (hero_id, hero_name, sidekick_name, sidekick_power)
    VALUES (?, ?, ?, ?)
''', sidekick_data)

query5 = "SELECT sidekick_name, sidekick_power, hero_name FROM sidekicks;"
sidekicks_df = pd.read_sql(query5, conn)
print("\nAll Sidekicks and Their Powers:")
print(sidekicks_df)

# Commit and close connection
conn.commit()
conn.close()





All Sidekicks and Their Powers:
   sidekick_name         sidekick_power       hero_name
0  Miles Morales           Web-slinging      Spider-Man
1    War Machine          Powered Armor        Iron Man
2          Robin             Acrobatics          Batman
3      Supergirl         Super Strength        Superman
4  Beta Ray Bill         God of Thunder            Thor
5     Donna Troy     Amazonian Strength    Wonder Woman
6          Shuri  Enhanced Intelligence   Black Panther
7      Kid Flash            Super Speed       The Flash
8       She-Hulk         Super Strength            Hulk
9           Wong            Mystic Arts  Doctor Strange
Sidekicks table successfully created.


## Bonus Challenge (+5 Points)

### Tasks:
1. Enhance your project by adding a visual or interactive component.

**Options:**
- Visualize battles using Matplotlib (e.g., a bar chart showing how many battles each superhero has fought).
- Make it interactive (e.g., allow users to input a superhero name and display their battles).

✅ **Checkpoint:** If attempting the bonus, ensure that your visualization or interactive component works correctly.

## Submission Guidelines

Each group must submit a **Google Colab Notebook (.ipynb)** containing:
- ✅ **Database schema (DDL)**
- ✅ **CSV imports & data insertion (DML)**
- ✅ **SQL queries & results**
- ✅ **Custom feature (bonus if applicable)**

## Grading Criteria (Total: 25 Points)

| Section                 | Points |
|-------------------------|--------|
| Database Setup          | 5      |
| Data Importing         | 10     |
| Querying the Data      | 10     |
| Extra Feature          | 5      |
| Bonus Challenge (Optional) | +5  |

### Final Notes:
- Be **creative** when designing superheroes and villains.
- **Comment your code** to explain its functionality.
- **Have fun** building your superhero universe!